In [1]:
!pip install openai
!pip install pandas
!pip install tqdm

In [2]:
import openai
import pandas as pd
from tqdm.notebook import tqdm

### Get an API Key from OpenAI

1. Go to https://platform.openai.com/ 
2. Sign up, add some credits, and generate a secure API key.
3. Copy and paste the key below.  Do not share the key with anyone!

In [3]:
openai.api_key='YOUR_API_KEY_HERE'

In [4]:
#These are the classes to match against.  If you change the prompt, change these as well
classes = ["Office expense", "Utilities", "Meals", "Travel", "Services"]

In [5]:
df = pd.read_csv('my-expenses-example.csv')

In [6]:
df

,Transaction Date,Posted Date,Card No.,Description,Debit
0,12/5/22,12/5/22,5555,Amazon Web Services AWS.Amazon.com WA,1.23
1,12/5/22,12/5/22,5555,EXPEDIA 722589820440EXPEDIA.COM WA,9.14
2,12/5/22,12/5/22,5555,EXPEDIA.COM TRAVEL SEATTLE WA,3.14
3,12/5/22,12/5/22,5555,EXPEDIA.COM TRAVEL SEATTLE WA,22.22
4,12/2/22,12/3/22,5555,AplPay STAPLES 01864ROCHESTER NY,33.33
5,12/2/22,12/3/22,5555,USPS PO 3571160610 0ROCHESTER NY,44.44


In [7]:
#Gets the text to use in the prompt
#This should be updated to match your spreadsheet
#If you have multiple text columns, experiment with concatenating them
def get_expense_text(df,idx):
    r = df.loc[idx]
    desc = r['Description']
    if  desc and not pd.isna(desc) and len(desc):
        #Only classify expenses with a description
        return f"{desc}"
    else:
        return None

#Try it out:
print(get_expense_text(df,0))

Amazon Web Services AWS.Amazon.com      WA


In [8]:
# Sends a prompt to OpenAI to classify an expense text.
# Read and understand the userprompt template, it required some 'prompt engineering'
def classify(text):
    userprompt = f"""For the expense classifications {', '.join(classes)}, which would you choose for the following expense description? Only respond with the expense verbatim with no punctuation."""
    prompt = f"{userprompt}\n'{text}'"
    #print(prompt)
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content":  prompt}
        ],
        max_tokens=50
    )
    
    return res

In [9]:
# Fuzzy matches a GPT response with a list of accepted classifications
def pluck(res):
    content = res.choices[0]["message"]["content"]
    if content:
        msg = content.strip().lower()
        for c in classes:
            if c.lower() in msg or msg in c.lower():
                return c
        return "Unknown" #Default to unknown classification
    return "INVALID MESSAGE"

In [10]:
#Classify the first expense in the spreadsheet
txt0 = get_expense_text(df,0)
print(txt0)
res0 = classify(txt0)
print(res0)
classified0 = pluck(res0)
print(classified0)

Amazon Web Services AWS.Amazon.com      WA
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Services",
        "role": "assistant"
      }
    }
  ],
  "created": 1687725717,
  "id": "chatcmpl-7VQqTtKTBvDJlYd3tWqDIGUIntdn0",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 56,
    "total_tokens": 57
  }
}
Services


In [11]:
#Classify each expense
classifications = []
for idx in tqdm(range(len(df))):
    text = get_expense_text(df,idx)
    if text and len(text):
        res = classify(text)
        val = pluck(res)
        classifications.append(val)
    else:
        classifications.append(None)

  0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
#Add the classifications as a new column to the dataframe
df['gpt_classification'] = classifications
df

,Transaction Date,Posted Date,Card No.,Description,Debit,gpt_classification
0,12/5/22,12/5/22,5555,Amazon Web Services AWS.Amazon.com WA,1.23,Services
1,12/5/22,12/5/22,5555,EXPEDIA 722589820440EXPEDIA.COM WA,9.14,Travel
2,12/5/22,12/5/22,5555,EXPEDIA.COM TRAVEL SEATTLE WA,3.14,Travel
3,12/5/22,12/5/22,5555,EXPEDIA.COM TRAVEL SEATTLE WA,22.22,Travel
4,12/2/22,12/3/22,5555,AplPay STAPLES 01864ROCHESTER NY,33.33,Office expense
5,12/2/22,12/3/22,5555,USPS PO 3571160610 0ROCHESTER NY,44.44,Office expense


In [13]:
#Safe and you're done!
df.to_csv('my-expenses-example-classified.csv')